# Germany Car Accidents Analysis  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [9]:
df = pd.read_csv('file/New_table_csv.csv')
df.head(800)




,Year,Months,Location,Personal Injury Accidents,Serious Accidents (Material Damage),Other Accidents (Intoxication),Other Accidents (Material Damage),Total Accidents
0,2011,January,Inside built-up areas,10595.0,3893.0,970.0,114745.0,130203.0
1,2011,January,Outside built-up areas (excl. motorways/freeways),4995.0,2137.0,181.0,31161.0,38474.0
2,2011,January,On motorways/freeways,1250.0,1094.0,45.0,8985.0,11374.0
3,2011,January,Total,16840.0,7124.0,1196.0,154891.0,180051.0
4,2011,February,Inside built-up areas,11008.0,3548.0,889.0,105064.0,120509.0
...,...,...,...,...,...,...,...,...
667,2024,November,Total,20679.0,5773.0,1143.0,186199.0,213794.0
668,2024,December,Inside built-up areas,NaN,NaN,NaN,NaN,NaN
669,2024,December,Outside built-up areas (excl. motorways/freeways),NaN,NaN,NaN,NaN,NaN
670,2024,December,On motorways/freeways,NaN,NaN,NaN,NaN,NaN


In [10]:
df.columns

Index(['Year', 'Months', 'Location', 'Personal Injury Accidents',
       'Serious Accidents (Material Damage)', 'Other Accidents (Intoxication)',
       'Other Accidents (Material Damage)', 'Total Accidents'],
      dtype='object')